In [14]:
import os
from argparse import Namespace

from pathlib2 import Path
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
# from pytorch_lightning.profiler import AdvancedProfiler
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.loggers.test_tube import TestTubeLogger

from mtmct_reid.data import ReIDDataModule
from mtmct_reid.engine import ST_ReID
from mtmct_reid.utils import save_args

import warnings

warnings.filterwarnings('ignore')

In [16]:
%cd './mtmct_reid/''

[WinError 3] The system cannot find the path specified: "'./mtmct_reid/''"
D:\Multi-Camera-Person-Re-Identification-master\Multi-Camera-Person-Re-Identification-master\mtmct_reid


In [7]:
%load_ext tensorboard
%tensorboard --logdir logs/

Launching TensorBoard...

KeyboardInterrupt: 

In [17]:
args = {
    "description": "Training and Predicting on Market-1501 Dataset",
    "data_dir": "../data/raw/Market-1501-v15.09.15",
    "log_path": "logs/",

    "learning_rate": 0.1,
    "criterion": "cross_entropy",
    "rerank": False,
    # "random_erasing": True,
    # "color_jitter": True,

    "gpus": 1,
    "max_epochs": 60,
    "precision": 16,
    "train_batchsize": 64,
    "val_batchsize": 64,
    "test_batchsize": 64,
    "limit_val_batches": 0.1,
    "profiler": True,
    "progress_bar_refresh_rate": 20
}

args = Namespace(**args)

In [11]:

tt_logger = TestTubeLogger(save_dir=args.log_path, name="",
                            description=args.description,
                            )

log_dir = Path(tt_logger.save_dir) / f"version_{tt_logger.version}"

checkpoint_dir = log_dir / "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
chkpt_callback = ModelCheckpoint(checkpoint_dir,
                                    monitor='Loss/val_loss',
                                    save_last=True,
                                    mode='min',
                                    save_top_k=10,
                                    )

data_module = ReIDDataModule.from_argparse_args(args)

model = ST_ReID(data_module.num_classes, learning_rate=args.learning_rate,
                criterion=args.criterion, rerank=args.rerank)

save_args(args, log_dir)

trainer = Trainer.from_argparse_args(args, logger=[tt_logger],
                                        checkpoint_callback=chkpt_callback,
                                        profiler=True)  # AdvancedProfiler()

# 
trainer.fit(model, data_module)

# Testing the loss & accuracy of the model
trainer.test(model)



Generating Spatial-Temporal Distribution.




Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\Dell/.cache\torch\hub\checkpoints\resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

MisconfigurationException: 
                You requested GPUs: [0]
                But your machine only has: []
            

In [18]:

!zip -r /content/experiment_results.zip /content/MTMCT-Person-Re-Identification/lightning_logs
from google.colab import files
files.download("/content/experiment_results.zip")

'zip' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'google.colab'